In [1]:
!nvidia-smi

Sat Jun 17 05:58:21 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install ultralytics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip install roboflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
from roboflow import Roboflow
rf = Roboflow(api_key="upclI1eGcbhbx5jxl1wv")
project = rf.workspace("360digitmg-vvros").project("inventory-of-steel-rods-um0a3")
dataset = project.version(9).download("yolov8")

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics<=8.0.20 is required but found version=8.0.118, to fix: `pip install ultralytics<=8.0.20`


Extracting Dataset Version Zip to Inventory-of-Steel-rods-9 in yolov8:: 100%|██████████| 4378/4378 [00:02<00:00, 1721.02it/s]


**Training**

In [6]:
!yolo task=detect mode=train model=yolov8x.pt data=/content/Inventory-of-Steel-rods-9/data.yaml epochs=5 imgsz=640 batch=4 project=/content/drive/MyDrive/yolov8/training_results name=inventory_train

100% 131M/131M [00:06<00:00, 20.1MB/s]
Ultralytics YOLOv8.0.118 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8x.pt, data=/content/Inventory-of-Steel-rods-9/data.yaml, epochs=5, patience=50, batch=4, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/yolov8/training_results, name=inventory_train, exist_ok=False, pretrained=False, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, class

# **Inferencing On Images**

In [8]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/yolov8/training_results/inventory_train3/weights/best.pt conf=0.55 source=/content/Inventory-of-Steel-rods-9/test/images, save_txt=True

WARNING ⚠️ 'source=/content/Inventory-of-Steel-rods-9/test/images,' does not require trailing comma ',', updating to 'source=/content/Inventory-of-Steel-rods-9/test/images'.
Ultralytics YOLOv8.0.118 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 268 layers, 68126457 parameters, 0 gradients

image 1/89 /content/Inventory-of-Steel-rods-9/test/images/16mm-115-_jpg.rf.6be1d46d77936559ae970154ea5eec49.jpg: 640x640 47 16 mms, 90.7ms
image 2/89 /content/Inventory-of-Steel-rods-9/test/images/16mm-135-_jpg.rf.b2e4be7ec295eb70c8b06922b5bff09c.jpg: 640x640 27 16 mms, 54.4ms
image 3/89 /content/Inventory-of-Steel-rods-9/test/images/16mm-145-_jpg.rf.467bf70f222a8ff993bdf54057c83586.jpg: 640x640 29 16 mms, 54.0ms
image 4/89 /content/Inventory-of-Steel-rods-9/test/images/16mm-147-_jpg.rf.dfc8108998fafc164dae6ed2acae7eff.jpg: 640x640 23 16 mms, 54.6ms
image 5/89 /content/Inventory-of-Steel-rods-9/test/images/16mm-233-_jpg.rf.852698881a349cb73757492a62e5d18a.jpg: 

In [9]:
import pandas as pd
import os
from glob import glob
class_dict = {
    0: "16 mm",
    1: "32 mm",
    2: "8 mm",
    # Add more class labels and their corresponding names as needed
}

folder = "/content/runs/detect/predict/labels"

# Get a list of all TXT files in the folder
txt_files = glob(os.path.join(folder, "*.txt"))

# Create an empty dictionary to store class counts
class_counts = {}

# Iterate over each TXT file
for txt_file in txt_files:
    # Read the current TXT file
    labels = pd.read_csv(txt_file, sep=' ', header=None)

    # Get class names
    labels['class'] = labels[0].map(class_dict)

    # Count number of detections for each class
    class_counts_per_file = labels['class'].value_counts().to_dict()

    # Update the overall class counts dictionary
    for cls, count in class_counts_per_file.items():
        class_counts[cls] = class_counts.get(cls, 0) + count

# Print the number of detections for each class
print("Number of detections for each class:")
for cls, count in class_counts.items():
    print(cls, count)

Number of detections for each class:
32 mm 3174
8 mm 5686
16 mm 587


In [10]:
!cp -r /content/runs/detect/predict /content/drive/MyDrive/yolov8/output

# **Export the model**

In [11]:
!yolo mode=export model=/content/drive/MyDrive/yolov8/training_results/inventory_train3/weights/best.pt format=onnx

Ultralytics YOLOv8.0.118 🚀 Python-3.10.12 torch-2.0.1+cu118 CPU
Model summary (fused): 268 layers, 68126457 parameters, 0 gradients

PyTorch: starting from /content/drive/MyDrive/yolov8/training_results/inventory_train3/weights/best.pt with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 7, 8400) (130.4 MB)
requirements: Ultralytics requirement "onnx>=1.12.0" not found, attempting AutoUpdate...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 277.1 MB/s eta 0:00:00

requirements: 1 package updated per ['onnx>=1.12.0']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.14.0 opset 17...
============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

ONNX: export success ✅ 18.